Mayar Ariss
25 Mar 2024

In [23]:
def generate_file(nod_names, xs, ys, zs, elm_names, grp_names, nod_name_lists, output_file):
    content = """\
#
analysis 3d.full dynamics
#
default.parameters
mass.damping.parameter = 19.1853
stiffness.damping.parameter = 0.0018
#
materials
 mat.name    model       properties
#  {{{1
                  # En,   ft,   fc,   Gf1,   Gf3,  mu
     math     mcri  1900   0.04    6.2  0.01    20    0.2 &
                   # Es,   co,   tanf,   Gf2
                     16090    0.23    0.58    0.1
#
                   # En,   ft,   fc,   Gf1,   Gf3,  mu
     matv     mcri  1140   0.7    3.1   0.1    20   0.2 &
                   # Es,   co,   tanf,   Gf2
                     166900    1.5    0.8    0.1
#
                   # Ed,   co_d,   tanf_d,  Uult(mm) 
     matd     mcrd   580   0.17   0.42    10.5  &
                   # Eem, py, pr, miou, miour, Epm
                     4570  28560.8  5710.36  0.0  0.0  -264.4437 &
                    #TOM spring - IP
                    #kpl1 Fy   ftu  cF alpha beta flag_strength Fult(%)     w
                     0.1 0.8  0.3 0.4  0.8  0.01       1        1        0 &
                    #TOM spring - OOP
                    #Uult(*ufmax) cF_OOP alpha_OOP beta_OOP
                        10        0.4     0.8      0.01 
#
#rigid base
                  # En,   ft,   fc,   Gf1,   Gf3,  mu
     math_r     mcri  5e5  100    1000  10    200    0.2 &
                   # Es,   co,   tanf,   Gf2
                     5e5    10 1 10   
#
                   # En,   ft,   fc,   Gf1,   Gf3,  mu
     matv_r     mcri 5e5  100    1000  10    200 0.2 &
                   # Es,   co,   tanf,   Gf2
                     5e5    10 1 10 
#
                   # Ed,   co_d,   tanf_d,  Uult(mm)
     matd_r    mcrd   5e8   100   1    1000   &
                   # Eem, py, pr, miou, miour, Epm
                     4570  28560.8  5710.36  0.0  0.0  -264.4437 &
                    #TOM spring - IP
                    #kpl1 Fy   ftu  cF alpha beta flag_strength Fult(%)     w
                     0.1 0.8  0.3 0.4  0.8  0.01       1        1        0 &
                    #TOM spring - OOP
                    #Uult(*ufmax) cF_OOP alpha_OOP beta_OOP
                        10        0.4     0.8      0.01 

#  }}}1
#
groups
# {{{1
#
type = mcr4
  grp.name    mat.names    thickness gauss.points edge.flags density ratio.of.mass mass.damping
    macro1  math  matv math  matv matd   250    9  3  9  3   1 1 1 1   1.9e-9    0.5    0
    macro2  math_r  matv_r math_r  matv_r matd   250    9  3  9  3   1 1 1 1   1.9e-9    0.5    0
#
type = cpl2
grp.name  stiffness.parameters
cpy        rigid rigid 0 0 0 0

type = sprd
grp.name       direction    spread.nodes   ratios.of.spread
 g_forc_x x 4 1.0 &
 1.0 1.0 1.0  

type = cnm3
grp.name      mass
m1         2.548
m2         5.096

#
#  }}}1
#
 structural.nodal
 nod.name           x            y            z
#   {{{1
"""

    # Add nodal coordinates
    for nod_name, x, y, z in zip(nod_names, xs, ys, zs):
        content += f"{nod_name:<10}{x:<12}{y:<12}{z:<12}\n"

    content += """\
#   }}}1
#
 element.connectivity
 elm.name       grp.name     nod.name
#   {{{
#
"""

    # Add element connectivity
    for elm_name, grp_name, nod_name_list in zip(elm_names, grp_names, nod_name_lists):
        content += f"{elm_name:<10}{grp_name:<12}{' '.join(map(str, nod_name_list))}\n"

    content += """\
#
# impose same vertical displacement on top nodes
101  cpy  6001004 6002001
102  cpy  6002001  6003001
103  cpy  6002001  6003001
#   }}}1
#
1000 g_forc_x &
  1000 6001004  6002001 6003001 6004001 

# Mass elements
  m1   m1  6001004 
  m2   m2  6002001
  m3   m2  6003001 
  m4   m1  6004001
  

 restraints
 nod.name      direction
       1001004  ry+rz
       1002001  ry+rz
       1002004  ry+rz
       1003001  ry+rz
       1003004  ry+rz
       
       6003002  rx+rz+z
       6004002  rx+rz+z
    #   
 1001004  x+y+z+rx+ry+rz
 1002001  x+y+z+rx+ry+rz
 1002004  x+y+z+rx+ry+rz
 1003001  x+y+z+rx+ry+rz
 1003004  x+y+z+rx+ry+rz
 1004001  x+y+z+rx+ry+rz
 #
 1000 y+z+rx+ry+rz
#
#
linear.curves
#   {{{1
 start.time = 0
 crv.name = c1
 file = Short_panel.acc.dat    
 first.line = 2
 #last.line = 1200
 #format = (f8.6,3x,f9.6)
#   }}}1
#
 applied.loading
 dynamic.loads
 nod.name       direction        type        crv.name          value 
#   {{{1
1000    x            a    c1   1.0
#   }}}1
 initial.loads
elm.name  type value
f 10201   udl1   0.0 -1.80e-5 0.0
r     1    -     0.0 0.0 0.0  2
r   100    -     0.0 0.0 0.0  2

nod.name       direction        type          value 
#   {{{1
  6001004        y            f    -25000.00000
  6002001        y            f    -50000.00000
  #6002004        y            f    -25000.00000
  6003001        y            f    -50000.00000
  #6003004        y            f    -25000.00000
  6004001        y            f    -25000.00000
#   }}}1
1000    x           v  -0.000615451

#
 equilibrium.stages
   end.of.stage    steps
    4000    4000
#
 solution
  solver = mumps
#
#dynamic analysis parameters
 integration.scheme
 scheme = hilber.hughes.taylor
  alpha = -0.33
#
 iterative.strategy
   number = 60
   initial.reformations = 60
   step.reduction = 60
   divergence.iteration = 60
   maximum.convergence =  1e7
#
 convergence.criteria
   tolerance = 1.000000e-05
   work.ref = 1.000000e+06
#
 output
 frequency 10 stress
#
end
"""
    # Write content to the output file
    with open(output_file, "w") as file:
        file.write(content)


In [24]:
# Example usage
nod_names = [1001003, 1001004, 1002001, 1002003, 1002004, 1003001, 1003003, 1003004, 1004001, 1004003]
xs = [0.0, 0.0, 333.33333, 333.33333, 333.33333, 666.66667, 666.66667, 666.66667, 1000.00000, 1000.00000]
ys = [250.0] * 10
zs = [0.0] * 10

elm_names = [10101, 10102, 10103]
grp_names = ['macro2', 'macro2', 'macro2']
nod_name_lists = [
    [1001004, 1002001, 1002003, 2002002, 2002001, 2001004, 2001002, 1001003],
    [1002004, 1003001, 1003003, 2003002, 2003001, 2002004, 2002002, 1002003],
    [1003004, 1004001, 1004003, 2004002, 2004001, 2003004, 2003002, 1003003]
]
generate_file(nod_names, xs, ys, zs, elm_names, grp_names, nod_name_lists, "generated_file.txt")